In [1]:
import pandas as pd
import datetime 
import numpy as np
import pickle
import matplotlib.pyplot as plt
import dtale
import re
import seaborn as sns

%matplotlib inline

In [2]:
truck = pd.read_csv('../data/average_truck_speeds_on_us_highways_2009.csv', skiprows=2)
#source: US Dept. of Transportation - https://ops.fhwa.dot.gov/freight/freight_analysis/nat_freight_stats/docs/10factsfigures/table3_8.htm

rail = pd.read_csv('../data/railroad_speed_data.csv')
#source: US Surface Transportation Board - https://www.stb.gov/reports-data/rail-service-data/

In [8]:
#truck.head()
#truck.info()
truck = truck[['Interstate Route', 'Average Operating Speed','Peak Period Average  Speed ','Non-Peak Period Average Speed']]


In [10]:
truck.head()
truck = truck.rename(columns ={
    'Interstate Route':'route',
    'Average Operating Speed':'avg_speed',
    'Peak Period Average  Speed ':'peak_avg_speed',
    'Non-Peak Period Average Speed':'non_peak_avg_speed'
})

In [11]:
truck.head()

,route,avg_speed,peak_avg_speed,non_peak_avg_speed
0,5,52.8,52.0,53.1
1,10,57.4,56.7,57.6
2,15,56.7,56.2,56.9
3,20,59.2,58.8,59.3
4,24,57.2,56.6,57.4


In [26]:
truck['avg_speed'].mean()

56.284

In [30]:
#truckspeed = truck['avg_speed'].mean().rename_axis('truck').reset_index(name='avg_mph_speed')

truckspeed = pd.DataFrame({
    "type" : ["truck"],
    "avg_mph_speed" : [56.284]
})

In [12]:
rail.head()

,Railroad/\nRegion,Category No.,Sub-Category,Measure,Variable,Sub-Variable,3/29/2017,4/5/2017,4/12/2017,4/19/2017,...,9/29/2021,10/6/2021,10/13/2021,10/20/2021,10/27/2021,11/3/2021,11/10/2021,11/17/2021,11/24/2021,12/1/2021
0,BNSF,1.0,NaN,Average Train Speed (MPH),Intermodal,NaN,32.7,31.6,32.3,32.5,...,33.0,33.2,33.5,33.2,33.0,32.4,31.8,32.3,32.0,33.8
1,BNSF,1.0,NaN,Average Train Speed (MPH),Grain unit,NaN,20.5,21.8,22.0,22.3,...,25.2,23.5,23.7,23.4,22.9,23.1,23.3,23.8,23.0,24.2
2,BNSF,1.0,NaN,Average Train Speed (MPH),Coal unit,NaN,24.1,24.1,23.6,23.4,...,22.9,21.7,22.4,22.3,22.1,21.9,22.6,22.3,22.6,24.1
3,BNSF,1.0,NaN,Average Train Speed (MPH),Automotive unit,NaN,24.9,24.7,24.8,26.5,...,25.1,26.7,26.6,27.2,26.3,25.9,25.1,26.0,24.8,26.3
4,BNSF,1.0,NaN,Average Train Speed (MPH),Crude oil unit,NaN,22.2,22.4,23.5,22.6,...,23.6,23.7,22.5,23.1,22.8,22.1,21.8,22.7,21.6,22.2


In [13]:
rail['Measure'].value_counts()

Grain Car Order Fulfillment Metrics (Count)                                                                  1192
Weekly total grain cars loaded and billed, reported by State and Type of Car Ordering System                  885
Weekly Carloads By 22 Commodity Categories                                                                    462
Weekly Average Number of Trains Holding Per Day by Train Type and Cause                                       220
Weekly Average Number of Loaded and Empty Cars in Revenue Service That Have Not Moved 48 Hours Or Greater     126
Average Terminal Dwell Time (Excluding Cars on Run Through Trains) (Hours)                                     78
Coal Unit Train Loadings or Carloadings by Coal Production Region (Count)                                      76
Cars On Line (Count)                                                                                           63
Average Train Speed  (MPH)                                                              

In [15]:
railspeed = rail.loc[rail.Measure =='Average Train Speed  (MPH)']

In [17]:
railspeed.head()
railspeed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56 entries, 0 to 2736
Columns: 251 entries, Railroad/
Region to 12/1/2021
dtypes: float64(1), object(250)
memory usage: 110.2+ KB


In [22]:
#I want the avg mph.  for knots, the conversion is 1.15

boat_plane_speed = pd.DataFrame({
    "type" : ["ship_slow", "ship_avg", "cargo_plane"],
    "avg_mph_speed" : ["22","26","560"]
})

#source: boats: https://transportgeography.org/contents/chapter4/transportation-and-energy/fuel-consumption-containerships/
#https://www.marineinsight.com/guidelines/speed-of-a-ship-at-sea/
#source: plane: https://www.flightdeckfriend.com/ask-a-pilot/how-fast-do-commercial-aeroplanes-fly

In [23]:
boat_plane_speed.head()

,type,avg_mph_speed
0,ship_slow,22
1,ship_avg,26
2,cargo_plane,560
